In [1]:
import pandas as pd
import numpy as np
import re
import json
import regex
import nltk # TODO: learnable POS encoder to add to model
from transformers import BertTokenizer
from sklearn.model_selection import train_test_split
from nltk import pos_tag
from transformers import AutoTokenizer
import tensorflow as tf

/home/ubuntu/Documents/CSE-143-Project-Fall-2024/env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-12-11 05:12:18.692858: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-11 05:12:18.707808: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733893938.726917   79524 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733893938.732401   79

In [2]:
# physical_devices = tf.config.experimental.list_physical_devices('GPU')
# if len(physical_devices) > 0:
#     print("We got a GPU")
#     tf.config.experimental.set_memory_growth(physical_devices[0], True)
# else:
#     print("Sorry, no GPU for you...")


We got a GPU


In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')

# Set memory growth to True for all GPUs (or False if you prefer not to use memory growth)
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)


In [3]:
nltk.download('averaged_perceptron_tagger_eng')


[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /home/ubuntu/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


True

In [4]:
# Read data
df_train = pd.read_csv('data/train.csv')
df_test = pd.read_csv('data/test.csv')

In [5]:
def clean_text(text):
    if not isinstance(text, str):
        text = str(text)
    text = re.sub(r'<.*?>', ' ', text)  # Remove HTML tags
    text = text.lower().strip()
    return text

def load_contractions(file_path="./contractions.json"):
    try:
        with open(file_path, 'r') as file:
            data = json.load(file)  # Load JSON data
        return data
    except (json.JSONDecodeError, FileNotFoundError):
        return {}

contractions = load_contractions()
contractions_re = re.compile(r'\b(' + '|'.join(re.escape(key) for key in contractions.keys()) + r')\b')

def expand_contractions(text):
    return contractions_re.sub(lambda x: contractions[x.group(0)], text)

def process_dataframe(frame):
    frame = frame.dropna(subset=['comment_text'])
    frame["comment_text"] = frame["comment_text"].apply(expand_contractions)
    return frame

TO_REMOVE = '"()+,-./:;<=>[\\]^_`{|}~\t\n“”’\'∞θ÷α•à−β∅³π‘₹´°£€\×™√²—'
OBSCENITY = '!#$%&*?@'

def remove_chars(text):
    pattern = f"[{re.escape(TO_REMOVE)}]"
    text = re.sub(pattern, " ", text)
    pattern = f"[{re.escape(OBSCENITY)}]"
    return re.sub(pattern, "", text)


In [6]:
df_train = process_dataframe(df_train)
df_test = process_dataframe(df_test)
df_train['comment_text'] = df_train['comment_text'].str.replace(r'\bhttp?\S+\b', 'link', regex=True)
df_test['comment_text'] = df_test['comment_text'].str.replace(r'\bhttp?\S+\b', 'link', regex=True)
df_train["comment_text"] = df_train["comment_text"].apply(clean_text)
df_test["comment_text"] = df_test["comment_text"].apply(clean_text)
df_train["comment_text"] = df_train["comment_text"].apply(remove_chars)
df_test["comment_text"] = df_test["comment_text"].apply(remove_chars)

/tmp/ipykernel_79524/3078383111.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  frame["comment_text"] = frame["comment_text"].apply(expand_contractions)


In [7]:
train_text = df_train["comment_text"].astype(str)
train_labels = df_train["target"].astype(float)
train_texts, val_texts, train_labels, val_labels = train_test_split(train_text, train_labels, test_size=0.1, random_state=7)


In [8]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased", use_fast=True)
print(tokenizer.is_fast)

# Function for POS tagging
def pos_tag_tokens(tokens, tokenizer):
    decoded = tokenizer.convert_ids_to_tokens(tokens)
    posTags = []
    for token in decoded:
        if token in ['[CLS]', '[SEP]', '[PAD]']:
            posTags.append((token, 'SPECIAL'))
        else:
            word = token.replace('##', '')
            posTags.append(pos_tag([word])[0])
    return posTags
tags = ['LS', 'TO', 'VBN', "''", 'WP', 'UH', 'VBG', 'JJ', 'VBZ', '--', 'VBP', 'NN', 'DT', 'PRP', ':', 'WP$', 'NNPS', 'PRP$', 'WDT', '(', ')', '.', ',', '``', '$', 'RB', 'RBR', 'RBS', 'VBD', 'IN', 'FW', 'RP', 'JJR', 'JJS', 'PDT', 'MD', 'VB', 'WRB', 'NNP', 'EX', 'NNS', 'SYM', 'CC', 'CD', 'POS']
taggerToId = {'SPECIAL': 0, 'NN': 1, 'VB': 2, 'JJ': 3, 'RB': 4, 'IN': 5}
maxInd = 5
for tag in tags:
    if tag not in taggerToId:
        maxInd += 1
        taggerToId[tag] = maxInd
# Function to convert tags to indices
def convertToInd(tags, taggerToId):
    return [taggerToId.get(tag, 0) for word, tag in tags]

# Prepare a tokenizer with batching
def batch_tokenize_and_tag(texts, tokenizer, batch_size=32, max_length=512):
    input_ids_list = []
    attention_masks_list = []
    pos_tag_list = []

    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i + batch_size]
        
        # Tokenize the batch with padding and truncation
        encoded_batch = tokenizer(
            batch_texts,
            truncation=True,  # Truncate to max length
            padding="max_length",  # Pad to max length
            max_length=max_length,  # Return tensors for TensorFlow
        )
        temp = tf.convert_to_tensor(encoded_batch['input_ids'])
        input_ids_list.append(temp)
        attention_masks_list.append(tf.convert_to_tensor(encoded_batch['attention_mask']))

        # POS tagging (convert tokenized ids to words and tag)
        for ids in temp:
            tokens = ids.numpy().tolist()
            tags = pos_tag_tokens(tokens, tokenizer)
            pos_tag_list.append(convertToInd(tags, taggerToId))
        print(f"Sample number {i}")
    # Concatenate everything into tensors
    input_ids = tf.concat(input_ids_list, axis=0)
    attention_masks = tf.concat(attention_masks_list, axis=0)
    # print(pos_tag_list)
    pos_tags_tensor = tf.convert_to_tensor(pos_tag_list, dtype=tf.int32)

    return {"input_ids": input_ids, "attention_mask": attention_masks, "pos_tags": pos_tags_tensor}

True


In [ ]:
# Tokenize and tag data in batches
train_data = batch_tokenize_and_tag(train_texts.tolist(), tokenizer, batch_size=256)
val_data = batch_tokenize_and_tag(val_texts.tolist(), tokenizer, batch_size=256)

# Output results
train_toks = {"input_ids": train_data["input_ids"], "attention_mask": train_data["attention_mask"]}
val_toks = {"input_ids": val_data["input_ids"], "attention_mask": val_data["attention_mask"]}
trainTagInds = train_data["pos_tags"]
valTagInds = val_data["pos_tags"]

I0000 00:00:1733894213.662597   79524 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9583 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:61:00.0, compute capability: 7.5
I0000 00:00:1733894213.663334   79524 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 9063 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:b1:00.0, compute capability: 7.5


Sample number 0
Sample number 256
Sample number 512
Sample number 768
Sample number 1024
Sample number 1280
Sample number 1536
Sample number 1792
Sample number 2048
Sample number 2304
Sample number 2560
Sample number 2816
Sample number 3072
Sample number 3328
Sample number 3584
Sample number 3840
Sample number 4096
Sample number 4352
Sample number 4608
Sample number 4864
Sample number 5120
Sample number 5376
Sample number 5632
Sample number 5888
Sample number 6144
Sample number 6400
Sample number 6656
Sample number 6912
Sample number 7168
Sample number 7424
Sample number 7680
Sample number 7936
Sample number 8192
Sample number 8448


In [ ]:
def save_tensor(tensor, path):
    tf.io.write_file(path, tf.io.serialize_tensor(tensor))
    return True
def load_tensor(path, dataType=tf.int32):
    return tf.io.parse_tensor(tf.io.read_file(path), out_type=dataType)

In [ ]:
save_tensor(train_toks["input_ids"], "Train_toks/train_input_ids")
save_tensor(train_toks["attention_mask"], "Train_toks/train_attention_mask")
save_tensor(val_toks["input_ids"], "Test_toks/test_input_ids")
save_tensor(val_toks["attention_mask"], "Test_toks/test_attention_mask")
save_tensor(trainTagInds, "Train_toks/train_tag_inds")
save_tensor(valTagInds, "Test_toks/test_tag_inds")


<tf.Tensor: shape=(256, 512), dtype=int32, numpy=
array([[ 0, 17, 15, ...,  0,  0,  0],
       [ 0, 38, 31, ...,  0,  0,  0],
       [ 0,  1,  1, ...,  0,  0,  0],
       ...,
       [ 0, 16,  1, ...,  0,  0,  0],
       [ 0,  4,  1, ...,  0,  0,  0],
       [ 0,  5, 16, ...,  0,  0,  0]], dtype=int32)>

46

In [12]:
from model import CombinedEmbeddingModel
num_secondary_embeddings = 46
embedding_dim = 16
dropout_rate = 0.3
model = CombinedEmbeddingModel(num_secondary_embeddings, embedding_dim, dropout_rate)


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [ ]:
# Train
train_labels = tf.convert_to_tensor(train_labels)
val_labels = tf.convert_to_tensor(val_labels)
loss = tf.keras.losses.BinaryCrossentropy(from_logits=False)
metrics = [tf.keras.metrics.BinaryAccuracy(), tf.keras.metrics.AUC()]
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss=loss, metrics=metrics)


In [14]:
val_ins = (val_toks["input_ids"], val_toks["attention_mask"], valTagInds)
train_ins = (train_toks["input_ids"], train_toks["attention_mask"], trainTagInds)
model.fit(x=train_ins, y=train_labels, validation_data=(val_ins, val_labels), epochs=3, batch_size = 64, verbose=1)

Epoch 1/3


I0000 00:00:1733893437.289493   75522 service.cc:148] XLA service 0x7f3ae800c680 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1733893437.289513   75522 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 2080 Ti, Compute Capability 7.5
I0000 00:00:1733893437.289516   75522 service.cc:156]   StreamExecutor device (1): NVIDIA GeForce RTX 2080 Ti, Compute Capability 7.5
2024-12-11 05:03:57.687377: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
W0000 00:00:1733893437.788395   75522 assert_op.cc:38] Ignoring Assert operator combined_embedding_model_1/tf_bert_model/bert/embeddings/assert_less/Assert/Assert
I0000 00:00:1733893438.586403   75522 cuda_dnn.cc:529] Loaded cuDNN version 90300
2024-12-11 05:04:01.530990: W external/local_xla/xla/service/gpu/nvptx_compiler.cc:930] The NVIDIA driver's CUDA version is 12.4 whic

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 742ms/step - auc: 0.5013 - binary_accuracy: 0.0231 - loss: 0.8050    

W0000 00:00:1733893446.959364   75524 assert_op.cc:38] Ignoring Assert operator combined_embedding_model_1/tf_bert_model/bert/embeddings/assert_less/Assert/Assert


4/4 ━━━━━━━━━━━━━━━━━━━━ 22s 3s/step - auc: 0.4913 - binary_accuracy: 0.0318 - loss: 0.7985 - val_auc: 0.5948 - val_binary_accuracy: 0.4648 - val_loss: 0.6760
Epoch 2/3
4/4 ━━━━━━━━━━━━━━━━━━━━ 6s 2s/step - auc: 0.5284 - binary_accuracy: 0.5891 - loss: 0.6523 - val_auc: 0.5745 - val_binary_accuracy: 0.6484 - val_loss: 0.5708
Epoch 3/3
4/4 ━━━━━━━━━━━━━━━━━━━━ 6s 2s/step - auc: 0.5144 - binary_accuracy: 0.6917 - loss: 0.5457 - val_auc: 0.5452 - val_binary_accuracy: 0.6484 - val_loss: 0.4930


In [15]:
# fine tune BERT 
model.bert.trainable=True
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), loss=loss, metrics=metrics)
model.fit(x=train_ins, y=train_labels, validation_data=(val_ins, val_labels), epochs=3, batch_size = 64, verbose=1)

Epoch 1/3


W0000 00:00:1733893635.154297   75524 assert_op.cc:38] Ignoring Assert operator combined_embedding_model_1/tf_bert_model/bert/embeddings/assert_less/Assert/Assert


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 738ms/step - auc: 0.5401 - binary_accuracy: 0.6722 - loss: 0.4723

W0000 00:00:1733893641.923856   75522 assert_op.cc:38] Ignoring Assert operator combined_embedding_model_1/tf_bert_model/bert/embeddings/assert_less/Assert/Assert


4/4 ━━━━━━━━━━━━━━━━━━━━ 17s 3s/step - auc: 0.5402 - binary_accuracy: 0.6725 - loss: 0.4732 - val_auc: 0.5448 - val_binary_accuracy: 0.6484 - val_loss: 0.4858
Epoch 2/3
4/4 ━━━━━━━━━━━━━━━━━━━━ 6s 2s/step - auc: 0.5430 - binary_accuracy: 0.6870 - loss: 0.4730 - val_auc: 0.5361 - val_binary_accuracy: 0.6484 - val_loss: 0.4790
Epoch 3/3
4/4 ━━━━━━━━━━━━━━━━━━━━ 6s 2s/step - auc: 0.5576 - binary_accuracy: 0.7010 - loss: 0.4630 - val_auc: 0.5440 - val_binary_accuracy: 0.6484 - val_loss: 0.4724


In [ ]:
model.save("combined_embedding_model_file")
loaded_model = tf.keras.load_model("combined_embedding_model_file")